### Dependencies

In [1]:
from rdflib import Graph
import owlrl

### Method to load data, perform reasoning and print query results

In [9]:
def queryLocalGraph(ontology_file, format_ontology, data_file, format_data, query_file):

    g = Graph()
    g.parse(data_file, format=format_data)#
    if ontology_file is not None: 
        g.parse(ontology_file, format=format_ontology)
    
    
    print("Loaded '" + str(len(g)) + "' triples.")
    
    #Performs OWL 2 RL  reasoning
    print("Performing reasoning (may take some time)...")
    owlrl.DeductiveClosure(owlrl.OWLRL_Semantics, axiomatic_triples=True, datatype_axioms=False).expand(g)
    
    print("After reasoning: '" + str(len(g)) + "' triples.")
    
    
    #Load query
    query = open(query_file, 'r').read()    
    
    qres = g.query(query)

    print("\nQuery: ")
    print(query)
    
    print("\nResults: ")
    
    #Print results
    for row in qres:        
        #Row is a list of matched RDF terms: URIs, literals or blank nodes
        row_str =""
        for element in row:
            row_str += str(element) + ", "
        
        print("'%s'" % (str(row_str))) 


### Playground dataset

In [3]:
#Playground
ontology_file=None
format_onto=None
dataset="./data/playground.ttl"
format_data = "ttl"
query="./data/query_playground.txt"

#Solution to be added
#query="solution/query7.1_playground.txt"
queryLocalGraph(ontology_file, format_onto, dataset, format_data, query)

Loaded '73' triples.
Performing reasoning (may take some time)...
After reasoning: '1238' triples.

Query: 
PREFIX ttr: <http://example.org/tuto/resource#>
PREFIX tto: <http://example.org/tuto/ontology#>
PREFIX dbp: <http://dbpedia.org/property/>
SELECT ?pet ?owner where {
?pet a tto:Animal .
OPTIONAL {?owner tto:pet ?pet}
}

Results: 
'http://example.org/tuto/resource#SnuffMonkey, None, '
'http://example.org/tuto/resource#LunaCat, http://example.org/tuto/resource#John, '
'http://example.org/tuto/resource#RexDog, http://example.org/tuto/resource#William, '
'http://example.org/tuto/resource#TomCat, http://example.org/tuto/resource#John, '


### World cities dataset

In [4]:
#World-cities
dataset = "./data/worldcities-free-100-task1.ttl"
format_data = "ttl"
ontology_file = "./data/ontology_worldcities.ttl"
format_onto = "ttl"
query = "./data/query_world-cities.txt"
queryLocalGraph(ontology_file, format_onto, dataset, format_data, query)

Loaded '1027' triples.
Performing reasoning (may take some time)...
After reasoning: '6018' triples.

Query: 
PREFIX wco: <http://www.semanticweb.org/ernesto/in3067-inm713/wco/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?country ?city ?pop WHERE {
	?city rdf:type wco:City .
    ?city wco:isCapitalOf ?country .
    ?city wco:population ?pop .
    FILTER (xsd:integer(?pop) > 5000000)
}
ORDER BY DESC(?pop)
LIMIT 10

Results: 
'http://www.semanticweb.org/ernesto/in3067-inm713/wco/japan, http://www.semanticweb.org/ernesto/in3067-inm713/wco/tokyo, 37977000, '
'http://www.semanticweb.org/ernesto/in3067-inm713/wco/indonesia, http://www.semanticweb.org/ernesto/in3067-inm713/wco/jakarta, 34540000, '
'http://www.semanticweb.org/ernesto/in3067-inm713/wco/philippines, http://www.semanticweb.org/ernesto/in3067-inm713/wco/manila, 23088000, '
'http://www.semanticweb.org/ernesto/in3067-inm713/wco/korea,_south, http://www.se

### Nobel prize dataset

In [10]:
#Nobel prize
#Reasoning takes a bit
ontology_file = "../files/nobel-prize-ontology.rdf"
format_onto = "xml"
dataset= "../files/nobelprize_kg.nt"
format_data = "nt"
query = "./data/query_nobel-prize.txt"

#Federation
#query = "./data/query_nobel-prize-service.txt";

queryLocalGraph(ontology_file, format_onto, dataset, format_data, query)


Loaded '85892' triples.
Performing reasoning (may take some time)...
After reasoning: '282253' triples.

Query: 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX np: <http://data.nobelprize.org/terms/>
SELECT DISTINCT ?name WHERE {
	?laur rdf:type np:Laureate . 
	?laur rdfs:label ?name .
	?laur foaf:gender "female" .
}
LIMIT 10

Results: 
'Marie Curie, née Sklodowska, '
'Maria Goeppert Mayer, '
'Irène Joliot-Curie, '
'Dorothy Crowfoot Hodgkin, '
'Gerty Theresa Cori, née Radnitz, '
'Rosalyn Yalow, '
'Barbara McClintock, '
'Rita Levi-Montalcini, '
'Gertrude B. Elion, '
'Christiane Nüsslein-Volhard, '


### SPARQL Update

In [5]:
dataset="./data/playground.ttl"
format_data = "ttl"
    
g = Graph()
g.parse(dataset, format=format_data)

print("Loaded '" + str(len(g)) + "' triples.")
  

Loaded '73' triples.


#### Insert triples

In [6]:
## Insert triples
query_insert = """
       PREFIX ttr: <http://example.org/tuto/resource#>\n
       PREFIX tto: <http://example.org/tuto/ontology#>\n
       PREFIX dbo: <http://dbpedia.org/ontology/>\n
       PREFIX dbp: <http://dbpedia.org/property/>\n
       insert data {
           ttr:Bella dbp:name 'Bella' .\n
           ttr:Bella a tto:Cat .\n
           ttr:Ernesto a dbo:Person .\n
           ttr:Ernesto ttr:pet ttr:Bella .\n                
        }"""
qres = g.update(query_insert)
print("Triples after insert update: '" + str(len(g)) + "' triples.")

Triples after insert update: '77' triples.


#### Remove triples

In [7]:
##Delete triples
query_delete = """
       PREFIX ttr: <http://example.org/tuto/resource#>\n
       PREFIX tto: <http://example.org/tuto/ontology#>\n
       PREFIX dbo: <http://dbpedia.org/ontology/>\n
       PREFIX dbp: <http://dbpedia.org/property/>\n
       delete data {
           ttr:Bella dbp:name 'Bella' .\n
           ttr:Bella a tto:Cat .\n
           ttr:Ernesto a dbo:Person .\n
           ttr:Ernesto ttr:pet ttr:Bella .\n                
        }"""
qres = g.update(query_delete)
print("Triples after delete update: '" + str(len(g)) + "' triples.")

Triples after delete update: '73' triples.


#### Update labels

In [8]:
##Update labels
query_update_labels = """
       PREFIX ttr: <http://example.org/tuto/resource#>\n
       PREFIX tto: <http://example.org/tuto/ontology#>\n
       PREFIX dbo: <http://dbpedia.org/ontology/>\n
       PREFIX dbp: <http://dbpedia.org/property/>\n
       PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\n
         delete { ?x dbp:name ?y }\n
         insert { ?x rdfs:label ?y }\n
         where { ?x dbp:name ?y }                
    """
qres = g.update(query_update_labels)
print("Triples after update labels: '" + str(len(g)) + "' triples.")
        
    
#Save new KG
file_output = dataset.replace(".ttl", "-update-labels.ttl")
g.serialize(destination=file_output, format='ttl')

Triples after update labels: '73' triples.


<Graph identifier=Nf1c15e9ef284408eb622f2c6e9103b81 (<class 'rdflib.graph.Graph'>)>